# Vector Search Images



## Import required libraries and environment variables

In [6]:
# Import libraries  
import os  
import json  
import requests  
from dotenv import load_dotenv  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
# from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import Vector  
# from azure.search.documents.indexes.models import (  
#     SearchIndex,  
#     SearchField,
#     SearchFieldDataType,  
#     SimpleField,
#     FieldMapping,
#     SearchableField,  
#     SearchIndex,  
#     VectorSearch,  
#     HnswVectorSearchAlgorithmConfiguration,  
#     SearchIndexerDataContainer,  
#     SearchIndexer,  
#     SearchIndexerDataSourceConnection,  
#     InputFieldMappingEntry,  
#     OutputFieldMappingEntry,  
#     SearchIndexerSkillset,
#     CorsOptions,
#     IndexingParameters,
#     IndexerStatus,
#     SearchIndexerDataContainer, SearchIndex, SearchIndexer, SimpleField, SearchFieldDataType,
#     EntityRecognitionSkill, InputFieldMappingEntry, OutputFieldMappingEntry, SearchIndexerSkillset,
#     CorsOptions, IndexingSchedule, SearchableField, IndexingParameters, SearchIndexerDataSourceConnection
# )  
# from azure.search.documents.indexes.models import WebApiSkill  
# from azure.storage.blob import BlobServiceClient  
# from azure.search.documents.indexes import SearchIndexerClient  
# from azure.search.documents.indexes.models import (  
#     SearchIndexerDataContainer,  
#     SearchIndexerDataSourceConnection,  
# )  
from IPython.display import Image, display 

  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")  
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")  
cogSvcsEndpoint = os.getenv("COGNITIVE_SERVICES_ENDPOINT")  
cogSvcsApiKey = os.getenv("COGNITIVE_SERVICES_API_KEY")  
customSkill_endpoint = os.getenv("FUNCTION_CUSTOM_SKILL_ENDPOINT")  
blob_connection_string = os.getenv("BLOB_CONNECTION_STRING")  
container_name = os.getenv("BLOB_CONTAINER_NAME")
credential = AzureKeyCredential(key)

## Perform a Vector search by vectorizing your text query

Perform a vector search to find the most relevant images based on the text query

In [9]:
# generate embeddings for the query
def generate_embeddings(text, cogSvcsEndpoint, cogSvcsApiKey):  
    url = f"{cogSvcsEndpoint}/computervision/retrieval:vectorizeText"  
  
    params = {  
        "api-version": "2023-02-01-preview"  
    }  
  
    headers = {  
        "Content-Type": "application/json",  
        "Ocp-Apim-Subscription-Key": cogSvcsApiKey  
    }  
  
    data = {  
        "text": text  
    }  
  
    response = requests.post(url, params=params, headers=headers, json=data)  
  
    if response.status_code == 200:  
        embeddings = response.json()["vector"]  
        return embeddings  
    else:  
        print(f"Error: {response.status_code} - {response.text}")  
        return None  

 

In [54]:


def search_image_by_text(query):
    # Initialize the SearchClient  
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
    vector = Vector(value=generate_embeddings(query, cogSvcsEndpoint, cogSvcsApiKey), k=3, fields="imageVector")  

    # Perform vector search  
    results = search_client.search(  
        search_text=None,  
        vectors=[vector],
        select=["title", "imageUrl"]  
    )   
    
    # Print the search results  
    for result in results:  
        print(f"Title: {result['title']}")  
        print(f"Image URL: {result['imageUrl']}") 
        print(f"Score: {result['@search.score']}")
        display(Image(url=result['imageUrl'], width=200)) 
        print("\n") 


In [40]:
print(result)

{'imageUrl': 'https://godzillastorage.blob.core.windows.net/searchvectordemo/Lighthouse.jpg', 'title': 'Lighthouse.jpg', '@search.score': 0.5556341, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}


search for:

* stay worm in winter
* camping 
* hunting
* frozen food

In [96]:
# read user input 
query = input("Enter your search query: ")
print(f"User search query: {query}\n")
search_image_by_text(query)


User search query: big hat

Title: Picture285.jpg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture285.jpg
Score: 0.611216




Title: Picture296.jpg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture296.jpg
Score: 0.5984226




Title: Picture999.jpeg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture999.jpeg
Score: 0.59392816


# Image to Image search
Perform a vector search to find the most relevant images based on the image query

In [56]:
def generate_embeddings_for_image(text, cogSvcsEndpoint, cogSvcsApiKey):  
    url = f"{cogSvcsEndpoint}/computervision/retrieval:vectorizeImage"  
  
    params = {  
        "api-version": "2023-02-01-preview"  
    }  
  
    headers = {  
        "Content-Type": "application/json",  
        "Ocp-Apim-Subscription-Key": cogSvcsApiKey  
    }  
  
    data = {  
        "url": text  
    }  
  
    response = requests.post(url, params=params, headers=headers, json=data)  
  
    if response.status_code == 200:  
        embeddings = response.json()["vector"]  
        return embeddings  
    else:  
        print(f"Error: {response.status_code} - {response.text}")  
        return None  

  


In [80]:
def search_image_by_image(query):
    # Initialize the SearchClient  
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
    vector = Vector(value=generate_embeddings_for_image(query, cogSvcsEndpoint, cogSvcsApiKey), k=3, fields="imageVector")  

    # Perform vector search  
    results = search_client.search(  
        search_text=None,  
        vectors=[vector],
        select=["title", "imageUrl"]  
    )  
    
    print(results)
    # Print the search results  
    for result in results:  
        print(f"Title: {result['title']}")  
        print(f"Image URL: {result['imageUrl']}") 
        print(f"Score: {result['@search.score']}")
        display(Image(url=result['imageUrl'], width=200)) 
        print("\n") 


In [78]:
# Generate text embeddings for the query (for context, this is a photo of a lady in a red hat)
queryimg = "https://images.unsplash.com/photo-1593476087123-36d1de271f08?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=387&q=80"  

# display image
display(Image(url=queryimg, width=200))


In [81]:
search_image_by_image(queryimg)

<iterator object azure.core.paging.ItemPaged at 0x2311cf7a0e0>
Title: Picture999.jpeg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture999.jpeg
Score: 0.93665427




Title: Picture290.jpg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture290.jpg
Score: 0.71684456




Title: Picture285.jpg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture285.jpg
Score: 0.7049761


In [88]:
queryimgs = [
    "https://img2.cgtrader.com/items/116754/1ecda16f39/lego-man-3d-model-max-obj.jpg",
    "https://www.team-bhp.com/forum/attachments/indian-car-scene/344291d1273211240-indian-police-cars-gcp_patrol_car.jpg",
    "https://godzillastorage.blob.core.windows.net/searchvectordemo/lego-policeman02.JPG", 
    # "https://wallpapercave.com/wp/e3J9loC.jpg", 
    # "https://i.pinimg.com/originals/ac/31/ea/ac31ea0cccac2424a6f03e8b9fcca71d.jpg", 
    # "https://img.washingtonpost.com/rf/image_1024w/2010-2019/WashingtonPost/2014/07/02/Others/Images/2014-07-02/emperor%20penguins%200011404339070.jpg", 
    # "http://3.bp.blogspot.com/-FrLvPDFZYwQ/TzoV4hQV8qI/AAAAAAAAGmQ/k0431I5GQNU/s1600/1.jpg"
]


for queryimg in queryimgs:
    # display image
    print("Search by image: ")
    display(Image(url=queryimg, width=200))
    print("Search Results:")
    search_image_by_image(queryimg)


Search by image: 


Search Results:
<iterator object azure.core.paging.ItemPaged at 0x2311d278d90>
Title: lego-policeman02.JPG
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/lego-policeman02.JPG
Score: 0.71520734




Title: Picture285.jpg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture285.jpg
Score: 0.63039386




Title: Picture621.jpg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Picture621.jpg
Score: 0.6265049




Search by image: 


...



Title: R.jpeg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/R.jpeg
Score: 0.7188035




Search by image: 


Search Results:
<iterator object azure.core.paging.ItemPaged at 0x2311ce1fc10>
Title: lego-policeman02.JPG
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/lego-policeman02.JPG
Score: 0.99999857




Title: R.jpeg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/R.jpeg
Score: 0.67286956




Title: Dubai Police 1.jpg
Image URL: https://godzillastorage.blob.core.windows.net/searchvectordemo/Dubai%20Police%201.jpg
Score: 0.65718144
